In [1]:
# basics
import numpy as np
import pandas as pd
import datetime
import re
import os,glob
import pickle
import textwrap

# statistics
import statsmodels.api as sm
import statsmodels.formula.api as smf


# plotting
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
matplotlib.rcParams['figure.figsize'] = [15,12]

# own data wrappers
from imp import reload
import measureclass as mc; reload(mc);
import coronadataclass as cdc; reload(cdc);
import crossvalidationclass as cvc; reload(cvc);

np.seterr(divide = 'ignore');


In [2]:
cccsl     = mc.COVID19_measures(datasource = 'CSH',       download_data = True)
oxford    = mc.COVID19_measures(datasource = 'OXFORD',    download_data = True)
whophsm   = mc.COVID19_measures(datasource = 'WHOPHSM',   download_data = True)
acaps     = mc.COVID19_measures(datasource = 'ACAPS',     download_data = True)
coronanet = mc.COVID19_measures(datasource = 'CORONANET', download_data = True)
hitcovid  = mc.COVID19_measures(datasource = 'HITCOVID',  download_data = True, dateformat = '%Y-%m-%d')

In [5]:
print('           {:8s} {:9s}'.format('Measures','Countries'))
print('CSH:       {:8d} {:9d}'.format(cccsl.MeasureList(mincount = 5).shape[0],     len(cccsl.countrylist)))
print('OXFORD:    {:8d} {:9d}'.format(oxford.MeasureList(mincount = 5).shape[0],    len(oxford.countrylist)))
print('WHOPHSM:   {:8d} {:9d}'.format(whophsm.MeasureList(mincount = 5).shape[0],   len(whophsm.countrylist)))
print('ACAPS:     {:8d} {:9d}'.format(acaps.MeasureList(mincount = 5).shape[0],     len(acaps.countrylist)))
print('CORONANET: {:8d} {:9d}'.format(coronanet.MeasureList(mincount = 5).shape[0], len(coronanet.countrylist)))
print('HITCOVID:  {:8d} {:9d}'.format(hitcovid.MeasureList(mincount = 5).shape[0],  len(hitcovid.countrylist)))


           Measures Countries
CSH:             55        57
OXFORD:          17       178
WHOPHSM:         41       213
ACAPS:           34       194
CORONANET:      109       198
HITCOVID:        39       136


In [7]:
cccsl.ImplementationTable(country = 'Austria', mincount= 5)

,ActivateCaseNotification,ActivateOrEstablishEmergencyResponse,ActivelyCommunicateWithManagers,AdaptProceduresForPatientManagement,AirportHealthCheck,AirportRestriction,BorderHealthCheck,BorderRestriction,ClosureOfEducationalInstitutions,CordonSanitaire,...,MeasuresToEnsureSecurityOfSupply,NationalLockdown,PersonalProtectiveMeasures,PublicTransportRestriction,Quarantine,ReOpeningOfEducationalInstitutions,ReturnOperationOfNationals,SmallGatheringCancellation,TracingAndTracking,TravelAlertAndWarning
22/01/2020,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
23/01/2020,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
24/01/2020,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25/01/2020,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
26/01/2020,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30/07/2020,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
31/07/2020,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
01/08/2020,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
02/08/2020,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [8]:
cccsl.MeasureList(mincount = 5)

,Measure_L1,Measure_L2,Countries with Implementation
ActivateCaseNotification,"Case identification, contact tracing and relat...",Activate case notification,20
AirportHealthCheck,"Case identification, contact tracing and relat...",Airport health check,31
BorderHealthCheck,"Case identification, contact tracing and relat...",Border health check,25
EnhanceDetectionSystem,"Case identification, contact tracing and relat...",Enhance detection system,28
IsolationOfCases,"Case identification, contact tracing and relat...",Isolation of cases,24
Quarantine,"Case identification, contact tracing and relat...",Quarantine,44
RestrictedTesting,"Case identification, contact tracing and relat...",Restricted testing,6
Surveillance,"Case identification, contact tracing and relat...",Surveillance,20
TracingAndTracking,"Case identification, contact tracing and relat...",Tracing and tracking,34
EnvironmentalCleaningAndDisinfection,Environmental measures,Environmental cleaning and disinfection,25


In [22]:
# generate a full table for a database

def AddDF(df1,df2):
    if not df1 is None:
        if df2 is None:
            return df1
        else:
            return pd.concat([df1,df2], ignore_index = True, sort = False)
    else:
        return df2
    

finaltable = None
for country in cccsl.countrylist:
    country_implementation_table = cccsl.ImplementationTable(country = country, mincount = 5).reset_index().rename(columns = {'index':'Date'})
    country_implementation_table['country'] = country
    finaltable = AddDF(finaltable, country_implementation_table)
finaltable.fillna(0, inplace = True)

finaltable

,Date,AdaptProceduresForPatientManagement,AirportHealthCheck,AirportRestriction,BorderRestriction,ClosureOfEducationalInstitutions,CrisisManagementPlans,EducateAndActivelyCommunicateWithThePublic,EnhanceDetectionSystem,EnhanceLaboratoryTestingCapacity,...,ReceiveInternationalHelp,PortAndShipRestriction,RepurposeHospitals,Research,Surveillance,MeasuresForPublicTransport,TheGovernmentProvideAssistanceToVulnerablePopulations,WorkSafetyProtocols,AccessToNonEssentialcriticalHealthcareServices,ExemptionOfQuarantine
0,22/01/2020,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,23/01/2020,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,24/01/2020,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,25/01/2020,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,26/01/2020,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11110,30/07/2020,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0
11111,31/07/2020,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0
11112,01/08/2020,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0
11113,02/08/2020,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0
